In [1]:
!pip install pandas
!pip install tqdm
!pip install sklearn
!pip install gensim==4.2.0

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [3]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import gensim

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
!nvidia-smi

Fri Oct  7 05:31:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:81:00.0 Off |                  N/A |
| 30%   31C    P8    19W / 350W |      3MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
def get_proauthors(row):
    return [i for i in row['authors'] if i < 100]

def get_coauthors(row):
    return [i for i in row['authors'] if i >= 100]

def has_proauthors(row):
    if len(row['proauthors']) > 0:
        return True
    else:
        return False

In [6]:
# data = json.load(open(r'data/backup/train.json'))
df = pd.read_json(r'data/backup/train.json')
df['proauthors'] = df.apply(lambda row: get_proauthors(row), axis=1)
df['coauthors'] = df.apply(lambda row: get_coauthors(row), axis=1)
df['has_proauthors'] = df.apply(lambda row: has_proauthors(row), axis=1)
df_f = df[df['has_proauthors']==False]
df_t = df[df['has_proauthors']==True]

In [7]:
df_t = resample(df_t, replace=True, n_samples=int(len(df_f)), random_state=90051)
# df_f = resample(df_f, replace=True, n_samples=int((1/3)*len(df_t)), random_state=90051)
df = pd.concat([df_t, df_f])

In [8]:
labels = []

for ids in df['proauthors']:
    label = np.zeros(101)
    if len(ids) > 0:
        label[ids] = 1.
    else:
        label[-1] = 1.
    labels.append(label)
    
labels = np.stack(labels, axis=0)

In [9]:
data = []

d2v = gensim.models.doc2vec.Doc2Vec.load(r'doc2vec.model')

for row in df.itertuples():
    if (row.Index%10000==0):
        print(row.Index)
        
    text = [str(i) for i in row.title]
    text.extend([str(i) for i in row.abstract])
    text =d2v.infer_vector(text)
    
    coauthors = np.zeros(21146)
    coauthors[[i-100 for i in row.coauthors]] = 1.
    
    venue = np.zeros(465)
    venue[[row.venue] if row.venue != '' else []] = 1.
    
    data.append(np.concatenate([text, coauthors, venue], axis=0))
    
data = np.stack(data)

0
10000
10000
0
10000
10000
0
20000


In [10]:
data.shape

(36666, 21867)

In [11]:
labels.shape

(36666, 101)

In [12]:
data_train, data_valid, labels_train, labels_valid = train_test_split(data, labels, test_size=0.2, random_state=90051)

In [13]:
train_set = TensorDataset(torch.tensor(data_train, dtype=torch.float), torch.tensor(labels_train, dtype=torch.float))
valid_set = TensorDataset(torch.tensor(data_valid, dtype=torch.float), torch.tensor(labels_valid, dtype=torch.float))

In [14]:
class AuthorAttriClf(nn.Module):
    def __init__(self):
        super(AuthorAttriClf, self).__init__()
        
        self.clf_block = nn.Sequential(
            nn.Linear(21867, 2048),
            nn.Dropout(),
            nn.Linear(2048, 1024),
            nn.Dropout(),
            nn.Linear(1024, 101),
        )

    def forward(self, input):
        probs = self.clf_block(input)

        return probs    

In [15]:
def train(train_status, model, optim, scheduler, criterion, epoch_size, train_loader, valid_loader):
   
    for epoch in range(epoch_size):
        model.train()
        epoch_loss = 0
        epoch_labels = torch.Tensor([])
        epoch_preds = torch.Tensor([])

        train_loop = tqdm(enumerate(train_loader), total=len(train_loader))
        train_loop.set_description(f"Epoch [{epoch+1}/{epoch_size}]")

        for batch, (inputs, labels) in train_loop:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optim.zero_grad()
            loss.backward()
            optim.step() 

            epoch_loss += loss.item()
            epoch_preds = torch.cat(((epoch_preds, (outputs.detach().cpu() > 0.5).int())), 0)
            epoch_labels = torch.cat((epoch_labels, labels.detach().cpu()), 0)

            train_loop.set_postfix_str(
                'train_loss={:.5f}'.format(loss.item())
            )

            if batch == len(train_loader)-1:
                epoch_loss /= len(train_loader.dataset)/train_loader.batch_size
                train_f1 = f1_score(epoch_labels, epoch_preds, average='samples', zero_division=1)
                valid_f1 = validate(model, valid_loader)
                train_loop.set_postfix_str(
                    'train_loss={:.5f}, train_f1={:.5f}, valid_f1={:.5f}'.format(
                        epoch_loss, train_f1, valid_f1
                    )
                )

                scheduler.step()

def validate(model, valid_loader):
    model.eval()
    valid_labels = torch.Tensor([])
    valid_preds = torch.Tensor([])
    with torch.no_grad():
        for batch, (inputs, labels) in enumerate(valid_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)

            valid_preds = torch.cat(((valid_preds, (outputs.detach().cpu() > 0.5).int())), 0)
            valid_labels = torch.cat((valid_labels, labels.detach().cpu()), 0)

    return f1_score(valid_labels, valid_preds, average='samples', zero_division=1)
    

In [ ]:
epoch_size = 50
batch_size = 32
lr = 1e-3

model = AuthorAttriClf().to(device)
optim = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optim, lr_lambda=lambda epoch: 0.96)
criterion = nn.BCEWithLogitsLoss()

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)

train_status = {'train_loss': []}
train(train_status, model, optim, scheduler, criterion, epoch_size, train_loader, valid_loader)

Epoch [6/50]:  94%|█████████▍| 866/917 [00:12<00:00, 60.37it/s, train_loss=0.00678]

In [ ]:
df_test = pd.read_json(r'data/backup/test.json')

pred_ids = []
for i in df_test['identifier']:
    pred_ids.append(i)
pred_ids = np.array(pred_ids)

data_test = []
for _, row in df_test.iterrows():
    text = [str(i) for i in row['title']]
    text.extend([str(i) for i in row['abstract']])
    text =d2v.infer_vector(text)
    
    coauthors = np.zeros(21146)
    coauthors[[i-100 for i in row['coauthors']]] = 1.
    
    venue = np.zeros(465)
    venue[[row['venue']] if row['venue'] != '' else []] = 1.
    
    data_test.append(np.concatenate([text, coauthors, venue], axis=0))
data_test = np.stack(data_test)

test_set = TensorDataset(torch.tensor(data_test, dtype=torch.float), torch.tensor(pred_ids, dtype=torch.float))
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=0)

In [ ]:
def get_predictions(model, test_loader, FILENAME="/content/gdrive/MyDrive/Colab Notebooks/COMP90051/Project2/data/pred.csv"):
    with torch.no_grad():
        preds = []
        identifiers = []

        for batch, (inputs, ids) in enumerate(test_loader):
            inputs = inputs.to(device)
            outputs = torch.sigmoid(model(inputs))

            for i in range(outputs.shape[0]):
                identifiers.append(int(ids[i]))
                pred = torch.nonzero((outputs[i].cpu() > 0.5).int())
                if len(pred[:-1]) > 0:
                    preds.append(" ".join([str(int(i)) for i in pred[:-1]]))
                else:
                    preds.append("-1")

        df = pd.DataFrame({'ID': identifiers, 'Predict': preds})
        df.to_csv(r'data/pred.csv', sep=',', index=False, encoding='utf-8')

get_predictions(model, test_loader)